# Data Quality               

## Step 1 

In [8]:
from typing import Any, Dict, Union, List
from dataclasses import dataclass
from datetime import datetime

import pandas as pd


@dataclass
class Metric:
    """Base class for Metric"""

    def __call__(self, df: pd.DataFrame) -> Dict[str, Any]:
        return {}


@dataclass
class CountTotal(Metric):
    """Total number of rows in DataFrame"""

    def __call__(self, df: pd.DataFrame) -> Dict[str, Any]:
        return {"total": len(df)}


@dataclass
class CountZeros(Metric):
    """Number of zeros in choosen column"""

    column: str

    def __call__(self, df: pd.DataFrame) -> Dict[str, Any]:
        n = len(df)
        k = sum(df[self.column] == 0)
        return {"total": n, "count": k, "delta": k / n}


@dataclass
class CountNull(Metric):
    """Number of empty values in choosen columns"""

    columns: List[str]
    aggregation: str = "any"  # either "all", or "any"

    def __call__(self, df: pd.DataFrame) -> Dict[str, Any]:
        n = len(df)
        
        mask = df[self.columns[0]].isna()
        if self.aggregation == "any":
            for column in self.columns[1:]:
                mask |= df[column].isna()
        else:
            for column in self.columns[1:]:
                mask &= df[column].isna()
        
        k = sum(mask)
        return {"total": n, "count": k, "delta": k / n}


@dataclass
class CountDuplicates(Metric):
    """Number of duplicates in choosen columns"""

    columns: List[str]

    def __call__(self, df: pd.DataFrame) -> Dict[str, Any]:
        n = len(df)
        k = sum(df.duplicated(subset=self.columns))
        return {"total": n, "count": k, "delta": k / n}


@dataclass
class CountValue(Metric):
    """Number of values in choosen column"""

    column: str
    value: Union[str, int, float]

    def __call__(self, df: pd.DataFrame) -> Dict[str, Any]:
        n = len(df)
        k = sum(df[self.column] == self.value)
        return {"total": n, "count": k, "delta": k / n}


@dataclass
class CountBelowValue(Metric):
    """Number of values below threshold"""

    column: str
    value: float
    strict: bool = False

    def __call__(self, df: pd.DataFrame) -> Dict[str, Any]:
        n = len(df)
        k = sum(df[self.column] < self.value if self.strict else df[self.column] <= self.value)
        return {"total": n, "count": k, "delta": k / n}


@dataclass
class CountBelowColumn(Metric):
    """Count how often column X below Y"""

    column_x: str
    column_y: str
    strict: bool = False

    def __call__(self, df: pd.DataFrame) -> Dict[str, Any]:
        n = len(df)
        k = sum(df[self.column_x] < df[self.column_y] if self.strict else df[self.column_x] <= df[self.column_y])
        return {"total": n, "count": k, "delta": k / n}


@dataclass
class CountRatioBelow(Metric):
    """Count how often X / Y below Z"""

    column_x: str
    column_y: str
    column_z: str
    strict: bool = False

    def __call__(self, df: pd.DataFrame) -> Dict[str, Any]:
        n = len(df)
        k = sum(df[self.column_x]/df[self.column_y] < df[self.column_z] if self.strict 
                else df[self.column_x]/df[self.column_y] <= df[self.column_z])
        return {"total": n, "count": k, "delta": k / n}


@dataclass
class CountCB(Metric):
    """Calculate lower/upper bounds for N%-confidence interval"""

    column: str
    conf: float = 0.95

    def __call__(self, df: pd.DataFrame) -> Dict[str, Any]:
        alpha = 1 - self.conf
        lcb, ucb = df[self.column].quantile([alpha/2, 1-alpha/2])
        return {"lcb": lcb, "ucb": ucb}


@dataclass
class CountLag(Metric):
    """A lag between latest date and today"""

    column: str
    fmt: str = "%Y-%m-%d"

    def __call__(self, df: pd.DataFrame) -> Dict[str, Any]:
        a = datetime.today ()
        b = datetime.strptime(max(df[self.column]), self.fmt)
        lag = a - b
        return {"today": a.strftime(self.fmt), "last_day": b.strftime(self.fmt), "lag": lag.days}


In [9]:
df_sales = pd.read_csv('ke_daily_sales.csv')
df_sales.head()

,day,item_id,qty,price,revenue
0,2022-10-24,100,5,120.0,500.0
1,2022-10-24,100,6,120.0,720.0
2,2022-10-24,200,2,200.0,400.0
3,2022-10-24,300,10,85.0,850.0
4,2022-10-23,100,3,110.0,330.0


In [10]:
df_visits = pd.read_csv('ke_visits.csv')
df_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   day       9 non-null      object
 1   item_id   9 non-null      int64 
 2   views     9 non-null      object
 3   clicks    9 non-null      int64 
 4   payments  9 non-null      int64 
dtypes: int64(3), object(2)
memory usage: 488.0+ bytes


In [11]:
ct_dict = {"total": 9}
ct = CountTotal()
assert ct(df_visits) == ct_dict

cz_dict = {"total": 9, "count": 1, "delta": 1 / 9}
cz = CountZeros(column='views')
assert cz(df_visits) == cz_dict

cn_dict = {'total': 9, 'count': 1, 'delta': 1/9}
df_visits.loc[2,'views'] = None
cn = CountNull(columns=['item_id', 'views'], aggregation="any")
assert cn(df_visits) == cn_dict

#для проверки поменять 'today' на текущую дату
cl_dict = {'today': '2023-10-09', 'last_day': '2022-10-24', 'lag': 350}
cl = CountLag(column ='day')
assert cl(df=df_sales) == cl_dict

AssertionError: 

## Step 2

In [12]:
"""Report checklist."""

from metrics import (
    CountTotal,
    CountLag,
    CountDuplicates,
    CountNull,
    CountRatioBelow,
    CountCB,
    CountZeros,
    CountBelowValue,
    CountBelowColumn,
)

# Checklist contains checks consist of:
# - table_name
# - metric
# - limits

CHECKLIST = [
    # Table with sales ["day", "item_id", "qty", "revenue", "price"]
    ("sales", CountTotal(), {"total": (1, 1e6)}),
    ("sales", CountLag("day"), {"lag": (0, 3)}),
    ("sales", CountDuplicates(["day", "item_id"]), {"total": (0, 0)}),
    ("sales", CountNull(["qty"]), {"total": (0, 0)}),
    ("sales", CountRatioBelow("revenue", "price", "qty", False), {"delta": (0, 0.05)}),
    ("sales", CountCB("revenue"), {}),
    ("sales", CountZeros("qty"), {"delta": (0, 0.3)}),
    ("sales", CountBelowValue("price", 100.0), {"delta": (0, 0.3)}),
    # Table with clickstream ["dt", "item_id", "views", "clicks", "payments"]
    ("relevance", CountTotal(), {"total": (1, 1e6)}),
    ("relevance", CountLag("dt"), {"lag": (0, 3)}),
    ("relevance", CountZeros("views"), {"delta": (0, 0.2)}),
    ("relevance", CountZeros("clicks"), {"delta": (0, 0.5)}),
    ("relevance", CountNull(["views", "clicks", "payments"]), {"delta": (0, 0.1)}),
    ("relevance", CountBelowValue("views", 10), {"delta": (0, 0.5)}),
    ("relevance", CountBelowColumn("clicks", "views"), {"total": (0, 0)}),
    ("relevance", CountBelowColumn("payments", "clicks"), {"total": (0, 0)}),
]

In [13]:
"""DQ Report."""

from typing import Dict, List, Tuple, Union, Callable
from dataclasses import dataclass
#from user_input.metrics import Metric

import pandas as pd

LimitType = Dict[str, Tuple[float, float]]
CheckType = Tuple[str, Metric, LimitType]



def memoize(func: Callable) -> Callable:
    """Memoize function"""
    cache = {}
    def wrapped(*argv, **kwargs):
        #print(cache)
        key = str(argv) + str(kwargs)
        if key not in cache:
            cache[key] = func(*argv, **kwargs)
        
        return cache[key]
    return wrapped

@dataclass
class Report:
    """DQ report class."""

    checklist: List[CheckType]
    engine: str = "pandas"
    
    @memoize
    def fit(self, tables: Dict[str, pd.DataFrame]) -> Dict:
        """Calculate DQ metrics and build report."""
        self.report_ = {}
        report = self.report_

        # Check if engine supported
        if self.engine != "pandas":
            raise NotImplementedError("Only pandas API currently supported!")
        
        result = []
        passed = 0
        failed = 0
        errors = 0
        for table_name, metric, limits in self.checklist:
            df = tables[table_name]
            error = ''
            try:
                values = metric(df)
                if len(limits) == 0:
                    status = '.'
                    passed+=1
                else:
                    # limits: {"total": (1, 1e6), "": () ... }
                    for column_limit, limit in limits.items():
                        value = values[column_limit]
                        if value >= limit[0] and value <= limit[1]:
                            status = '.'
                            passed+=1
                        else:
                            status = 'F'
                            failed+=1
                  
            except Exception as e:
                status = 'E'
                error = type(e).__name__ 
                errors+=1
        
            result.append({'table_name': table_name,
                           'metric': str(metric),
                           'limits': str(limits),
                           'values': values,
                           'status': status,
                           'error': error})
        
        
        report['title'] = 'DQ Report for tables ' + str(sorted(list(tables.keys())))
        report['result'] = pd.DataFrame(result)
        report['total'] = len(result)
        report['passed'] = passed
        report['passed_pct'] = round(passed / report['total'] * 100, 2)
        report['failed'] = failed
        report['failed_pct'] = round(failed / report['total'] * 100, 2)
        report['errors'] = errors
        report['errors_pct'] = round(errors / report['total'] * 100, 2)
        

        return report

    def to_str(self) -> None:
        """Convert report to string format."""
        report = self.report_

        msg = (
            "This Report instance is not fitted yet. "
            "Call 'fit' before using this method."
        )

        assert isinstance(report, dict), msg

        pd.set_option("display.max_rows", 500)
        pd.set_option("display.max_columns", 500)
        pd.set_option("display.max_colwidth", 20)
        pd.set_option("display.width", 1000)

        return (
            f"{report['title']}\n\n"
            f"{report['result']}\n\n"
            f"Passed: {report['passed']} ({report['passed_pct']}%)\n"
            f"Failed: {report['failed']} ({report['failed_pct']}%)\n"
            f"Errors: {report['errors']} ({report['errors_pct']}%)\n"
            "\n"
            f"Total: {report['total']}"
        )


In [44]:
cl = Report(CHECKLIST[0])

In [45]:
cl.fit({'sales': df_sales, 'relevance': df_visits})

ValueError: too many values to unpack (expected 3)

In [38]:
cl.to_str()

"DQ Report for tables ['relevance', 'sales']\n\n   table_name               metric               limits               values status     error\n0       sales         CountTotal()  {'total': (1, 10...         {'total': 7}      .          \n1       sales  CountLag(column=...      {'lag': (0, 3)}  {'today': '2023-...      F          \n2       sales  CountDuplicates(...    {'total': (0, 0)}  {'total': 7, 'co...      F          \n3       sales  CountNull(column...    {'total': (0, 0)}  {'total': 7, 'co...      F          \n4       sales  CountRatioBelow(...  {'delta': (0, 0....  {'total': 7, 'co...      F          \n5       sales  CountCB(column='...                   {}  {'lcb': 49.50000...      .          \n6       sales  CountZeros(colum...  {'delta': (0, 0.3)}  {'total': 7, 'co...      .          \n7       sales  CountBelowValue(...  {'delta': (0, 0.3)}  {'total': 7, 'co...      .          \n8   relevance         CountTotal()  {'total': (1, 10...         {'total': 9}      .          \n9 

# PySpark

Документация с примерами
- https://sparkbyexamples.com 

In [84]:
# Import the PySpark libraries
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext

# Create a SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()

# Create a SQLContext
sqlContext = SQLContext(spark)

# Load a CSV file
df = sqlContext.read.csv("ke_visits.csv", header=True)
df_relevance = sqlContext.read.csv("ke_daily_sales.csv", header=True)

# Count the number of rows in the DataFrame
rowCount = df.count()

# Print the row count
print("Number of rows:", rowCount)

from pyspark.sql.types import IntegerType
df = df.withColumn("views", df["views"].cast(IntegerType()))
df = df.withColumn("clicks", df["clicks"].cast(IntegerType()))
df = df.withColumn("item_id", df["item_id"].cast(IntegerType()))
df = df.withColumn("payments", df["payments"].cast(IntegerType()))

# Print the schema of the DataFrame
df.printSchema()

# Show the first 10 rows of the DataFrame
df.show(10)


Number of rows: 9
root
 |-- day: string (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- views: integer (nullable = true)
 |-- clicks: integer (nullable = true)
 |-- payments: integer (nullable = true)

+----------+-------+-----+------+--------+
|       day|item_id|views|clicks|payments|
+----------+-------+-----+------+--------+
|2022-09-24|    100| 1000|   219|      56|
|2022-09-24|    200| 1248|   343|       1|
|2022-09-24|    300|  993|   102|      71|
|2022-09-23|    100| 3244|   730|      18|
|2022-09-23|    200|  830|   203|       9|
|2022-09-23|    300|    0|     0|       2|
|2022-09-22|    100| NULL|   123|      20|
|2022-09-22|    200| 5320|   500|      13|
|2022-09-22|    300|  777|    68|       2|
+----------+-------+-----+------+--------+



In [85]:
from typing import Any, Dict, Union, List
from dataclasses import dataclass
from datetime import datetime

import pandas as pd
import pyspark.sql as ps


@dataclass
class Metric:
    """Base class for Metric"""

    def __call__(self, df: Union[pd.DataFrame, ps.DataFrame]) -> Dict[str, Any]:
        if isinstance(df, pd.DataFrame):
            return self._call_pandas(df)

        if isinstance(df, ps.DataFrame):
            return self._call_pyspark(df)

        msg = (
            f"Not supported type of arg 'df': {type(df)}. "
            "Supported types: pandas.DataFrame, "
            "pyspark.sql.dataframe.DataFrame"
        )
        raise NotImplementedError(msg)

    def _call_pandas(self, df: pd.DataFrame) -> Dict[str, Any]:
        return {}

    def _call_pyspark(self, df: ps.DataFrame) -> Dict[str, Any]:
        return {}


@dataclass
class CountTotal(Metric):
    """Total number of rows in DataFrame"""

    def _call_pandas(self, df: pd.DataFrame) -> Dict[str, Any]:
        return {"total": len(df)}

    def _call_pyspark(self, df: ps.DataFrame) -> Dict[str, Any]:
        return {"total": df.count()}


@dataclass
class CountZeros(Metric):
    """Number of zeros in choosen column"""

    column: str

    def _call_pandas(self, df: pd.DataFrame) -> Dict[str, Any]:
        n = len(df)
        k = sum(df[self.column] == 0)
        return {"total": n, "count": k, "delta": k / n}

    def _call_pyspark(self, df: ps.DataFrame) -> Dict[str, Any]:
        from pyspark.sql.functions import col, count

        n = df.count()
        k = df.filter(col(self.column) == 0).count()
        return {"total": n, "count": k, "delta": k / n}



@dataclass
class CountNull(Metric):
    """Number of empty values in choosen columns"""

    columns: List[str]
    aggregation: str = "any"  # either "all", or "any"
    
    def _call_pandas(self, df: pd.DataFrame) -> Dict[str, Any]:
        n = len(df)
        mask = df[self.columns[0]].isna()
        if self.aggregation == "any":
            for column in self.columns[1:]:
                mask |= df[column].isna()
        else:
            for column in self.columns[1:]:
                mask &= df[column].isna()
        
        k = sum(mask)
        return {"total": n, "count": k, "delta": k / n}

    def _call_pyspark(self, df: ps.DataFrame) -> Dict[str, Any]:
        from pyspark.sql.functions import col, count, isnan

        n = df.count()
        mask = col(self.columns[0]).isNull() |  isnan(col(self.columns[0]))
        if self.aggregation == "any":
            for column in self.columns[1:]:
                c = col(column)
                mask = mask | (c.isNull() |  isnan(c))
        else:
            for column in self.columns[1:]:
                c = col(column)
                mask = mask & (c.isNull() |  isnan(c))
        
        k = df.filter(mask).count()        
                
        return {"total": n, "count": k, "delta": k / n}

    

@dataclass
class CountDuplicates(Metric):
    """Number of duplicates in choosen columns"""

    columns: List[str]

    def _call_pandas(self, df: pd.DataFrame) -> Dict[str, Any]:
        n = len(df)
        k = sum(df.duplicated(subset=self.columns))
        return {"total": n, "count": k, "delta": k / n}
    
    def _call_pyspark(self, df: ps.DataFrame) -> Dict[str, Any]:
        from pyspark.sql.functions import col, count
        n = df.count()
        k = df.groupby(self.columns).count().where(col('count') > '1').count()
        return {"total": n, "count": k, "delta": k / n}


@dataclass
class CountValue(Metric):
    """Number of values in choosen column"""

    column: str
    value: Union[str, int, float]

    def _call_pandas(self, df: pd.DataFrame) -> Dict[str, Any]:
        n = len(df)
        k = sum(df[self.column] == self.value)
        return {"total": n, "count": k, "delta": k / n}
    
    def _call_pyspark(self, df: ps.DataFrame) -> Dict[str, Any]:
        from pyspark.sql.functions import col
        n = df.count()
        k = df.filter(col(self.column) == self.value).count()
        return {"total": n, "count": k, "delta": k / n}


@dataclass
class CountBelowValue(Metric):
    """Number of values below threshold"""

    column: str
    value: float
    strict: bool = False

    def _call_pandas(self, df: pd.DataFrame) -> Dict[str, Any]:
        n = len(df)
        k = sum(df[self.column] < self.value if self.strict else df[self.column] <= self.value)
        return {"total": n, "count": k, "delta": k / n}
    
    def _call_pyspark(self, df: ps.DataFrame) -> Dict[str, Any]:
        from pyspark.sql.functions import col
        n = df.count()
        if self.strict:   
            k = df.filter(col(self.column) < self.value).count()
        else:
            k = df.filter(col(self.column) <= self.value).count()
        return {"total": n, "count": k, "delta": k / n}


@dataclass
class CountBelowColumn(Metric):
    """Count how often column X below Y"""

    column_x: str
    column_y: str
    strict: bool = False

    def _call_pandas(self, df: pd.DataFrame) -> Dict[str, Any]:
        n = len(df)
        k = sum(df[self.column_x] < df[self.column_y] if self.strict else df[self.column_x] <= df[self.column_y])
        return {"total": n, "count": k, "delta": k / n}
    
    def _call_pyspark(self, df: ps.DataFrame) -> Dict[str, Any]:
        from pyspark.sql.functions import col, isnan
        n = df.count()
        mask_x = ~isnan(col(self.column_x)) & col(self.column_x).isNotNull()
        mask_y = ~isnan(col(self.column_y)) & col(self.column_y).isNotNull()
        if self.strict:   
            k = df.filter(mask_x & mask_y & (col(self.column_x) < col(self.column_y))).count()
        else:
            k = df.filter(mask_x & mask_y & (col(self.column_x) <= col(self.column_y))).count()
        return {"total": n, "count": k, "delta": k / n}


@dataclass
class CountRatioBelow(Metric):
    """Count how often X / Y below Z"""

    column_x: str
    column_y: str
    column_z: str
    strict: bool = False

    def _call_pandas(self, df: pd.DataFrame) -> Dict[str, Any]:
        n = len(df)
        k = sum(df[self.column_x]/df[self.column_y] < df[self.column_z] if self.strict 
                else df[self.column_x]/df[self.column_y] <= df[self.column_z])
        return {"total": n, "count": k, "delta": k / n}
    
    def _call_pyspark(self, df: ps.DataFrame) -> Dict[str, Any]:
        from pyspark.sql.functions import col, isnan
        n = df.count()
        ration = col(self.column_x)/col(self.column_y)
        mask_x = ~isnan(col(self.column_x)) & col(self.column_x).isNotNull()
        mask_y = ~isnan(col(self.column_y)) & col(self.column_y).isNotNull()
        mask_z = ~isnan(col(self.column_z)) & col(self.column_z).isNotNull()
        mask = mask_x & mask_y & mask_z
        if self.strict:   
            k = df.filter(mask & (ration < col(self.column_z))).count()
        else:
            k = df.filter(mask & (ration <= col(self.column_z))).count()
        return {"total": n, "count": k, "delta": k / n}



@dataclass
class CountCB(Metric):
    """Calculate lower/upper bounds for N%-confidence interval"""

    column: str
    conf: float = 0.95

    def _call_pandas(self, df: pd.DataFrame) -> Dict[str, Any]:
        alpha = 1 - self.conf
        lcb, ucb = df[self.column].quantile([alpha/2, 1-alpha/2])
        return {"lcb": lcb, "ucb": ucb}
    
    def _call_pyspark(self, df: ps.DataFrame) -> Dict[str, Any]:
        from pyspark.sql.functions import percentile_approx
        alpha = 1 - self.conf
        q = df.select(percentile_approx(self.column, [alpha/2, 1-alpha/2])).collect()[0][0]
        return {"lcb": q[0], "ucb": q[1]}


@dataclass
class CountLag(Metric):
    """A lag between latest date and today"""
    
    column: str
    fmt: str = "%Y-%m-%d"

    def _call_pandas(self, df: pd.DataFrame) -> Dict[str, Any]:
        a = datetime.today ()
        b = datetime.strptime(max(df[self.column]), self.fmt)
        lag = a - b
        return {"today": a.strftime(self.fmt), "last_day": b.strftime(self.fmt), "lag": lag.days}

    def _call_pyspark(self, df: pd.DataFrame) -> Dict[str, Any]:
        from pyspark.sql.functions import max
        a = datetime.today ()
        max_time = df.select(max(self.column)).collect()[0][0]
        b = datetime.strptime(max_time, self.fmt)
        lag = a - b
        return {"today": a.strftime(self.fmt), "last_day": b.strftime(self.fmt), "lag": lag.days}


In [86]:
#Проверки

df = df.replace([123, 1000], [None, None])
cn_dict = {'total': 9, 'count': 2, 'delta': 2/9}
cn = CountNull(columns=['clicks', 'views'], aggregation="any")
assert cn(df) == cn_dict

new_df = spark.createDataFrame([('2022-09-24', '200', '1248', '343', '1')], df.columns)
df = df.union(new_df)
df = df.withColumn("views", df["views"].cast(IntegerType()))
df = df.withColumn("clicks", df["clicks"].cast(IntegerType()))
df = df.withColumn("item_id", df["item_id"].cast(IntegerType()))
df = df.withColumn("payments", df["payments"].cast(IntegerType()))
cd_dict = {'total': 10, 'count': 1, 'delta': 1/10}
cd = CountDuplicates(['clicks', 'views'])
assert cd(df) == cd_dict


cv_dict = {'total': 10, 'count': 2, 'delta': 2/10}
cv = CountValue('clicks', 343)
assert cv(df) == cv_dict

cbv_dict = {'total': 10, 'count': 7, 'delta': 7/10}
cbv = CountBelowValue('clicks', 343)
assert cbv(df) == cbv_dict

cbc_dict = {'total': 10, 'count': 1, 'delta': 1/10}
cbc = CountBelowColumn('views', 'clicks')
assert cbc(df) == cbc_dict

ccb_dict = {'lcb': 0, 'ucb': 730}
ccb = CountCB('clicks')
assert ccb(df) == ccb_dict

cl_dict = {'today': '2023-10-20', 'last_day': '2022-09-24', 'lag': 391}
cl = CountLag(column ='day')
assert cl(df=df) == cl_dict

In [61]:
from typing import Dict, List, Tuple, Union, Callable
from dataclasses import dataclass
#from user_input.metrics import Metric

import pandas as pd
import pyspark.sql as ps

LimitType = Dict[str, Tuple[float, float]]
CheckType = Tuple[str, Metric, LimitType]


def memoize(func: Callable) -> Callable:
    """Memoize function"""
    cache = {}
    def wrapped(*argv, **kwargs):
        key = str(argv) + str(kwargs)
        if key not in cache:
            cache[key] = func(*argv, **kwargs)
        
        return cache[key]
    return wrapped


@dataclass
class Report:
    """DQ report class."""

    checklist: List[CheckType]
    engine: str = "pandas"
    
    
    def fit(self, tables: Dict[str, Union[pd.DataFrame, ps.DataFrame]]) -> Dict:
        """Calculate DQ metrics and build report."""

        if self.engine == "pandas":
            return self._fit_pandas(tables)

        if self.engine == "pyspark":
            return self._fit_pyspark(tables)

        raise NotImplementedError("Only pandas and pyspark APIs currently supported!")
    
    def _fit_pandas(self, tables: Dict[str, pd.DataFrame]) -> Dict:
        return self._fit(tables)

    def _fit_pyspark(self, tables: Dict[str, ps.DataFrame]) -> Dict:
        return self._fit(tables)
    
    
    @memoize
    def _fit(self, tables: Dict[str, Union[pd.DataFrame, ps.DataFrame]]) -> Dict:
        """Initial method fit"""
        self.report_ = {}
        report = self.report_

        result = []
        passed = 0
        failed = 0
        errors = 0
        for table_name, metric, limits in self.checklist:
            df = tables[table_name]
            error = ''
            try:
                values = metric(df)
                if len(limits) == 0:
                    status = '.'
                    passed+=1
                else:
                    # limits: {"total": (1, 1e6), "": () ... }
                    for column_limit, limit in limits.items():
                        value = values[column_limit]
                        if value >= limit[0] and value <= limit[1]:
                            status = '.'
                            passed+=1
                        else:
                            status = 'F'
                            failed+=1
                  
            except Exception as e:
                status = 'E'
                error = type(e).__name__ 
                errors+=1
        
            result.append({'table_name': table_name,
                           'metric': str(metric),
                           'limits': str(limits),
                           'values': values,
                           'status': status,
                           'error': error})
        
        
        report['title'] = 'DQ Report for tables ' + str(sorted(list(tables.keys())))
        report['result'] = pd.DataFrame(result)
        report['total'] = len(result)
        report['passed'] = passed
        report['passed_pct'] = round(passed / report['total'] * 100, 2)
        report['failed'] = failed
        report['failed_pct'] = round(failed / report['total'] * 100, 2)
        report['errors'] = errors
        report['errors_pct'] = round(errors / report['total'] * 100, 2)
        

        return report
    

    def to_str(self) -> None:
        """Convert report to string format."""
        report = self.report_

        msg = (
            "This Report instance is not fitted yet. "
            "Call 'fit' before using this method."
        )

        assert isinstance(report, dict), msg

        pd.set_option("display.max_rows", 500)
        pd.set_option("display.max_columns", 500)
        pd.set_option("display.max_colwidth", 20)
        pd.set_option("display.width", 1000)

        return (
            f"{report['title']}\n\n"
            f"{report['result']}\n\n"
            f"Passed: {report['passed']} ({report['passed_pct']}%)\n"
            f"Failed: {report['failed']} ({report['failed_pct']}%)\n"
            f"Errors: {report['errors']} ({report['errors_pct']}%)\n"
            "\n"
            f"Total: {report['total']}"
        )


In [62]:
cl = Report(CHECKLIST, 'pyspark')

In [65]:
cl.fit({'sales': df_relevance, 'relevance': df})
cl.to_str()

"DQ Report for tables ['relevance', 'sales']\n\n   table_name               metric               limits               values status              error\n0       sales         CountTotal()  {'total': (1, 10...         {'total': 7}      .                   \n1       sales  CountLag(column=...      {'lag': (0, 3)}  {'today': '2023-...      F                   \n2       sales  CountDuplicates(...    {'total': (0, 0)}  {'total': 7, 'co...      F                   \n3       sales  CountNull(column...    {'total': (0, 0)}  {'total': 7, 'co...      F                   \n4       sales  CountRatioBelow(...  {'delta': (0, 0....  {'total': 7, 'co...      E          NameError\n5       sales  CountCB(column='...                   {}  {'lcb': 0.0, 'uc...      .                   \n6       sales  CountZeros(colum...  {'delta': (0, 0.3)}  {'total': 7, 'co...      .                   \n7       sales  CountBelowValue(...  {'delta': (0, 0.3)}  {'total': 7, 'co...      .                   \n8   relevance   

In [74]:
df.show()

+----------+-------+-----+------+--------+
|       day|item_id|views|clicks|payments|
+----------+-------+-----+------+--------+
|2022-09-24|    100| NULL|   219|      56|
|2022-09-24|    200| 1248|   343|       1|
|2022-09-24|    300|  993|   102|      71|
|2022-09-23|    100| 3244|   730|      18|
|2022-09-23|    200|  830|   203|       9|
|2022-09-23|    300|    0|     0|       2|
|2022-09-22|    100| NULL|  NULL|      20|
|2022-09-22|    200| 5320|   500|      13|
|2022-09-22|    300|  777|    68|       2|
|2022-09-24|    200| 1248|   343|       1|
+----------+-------+-----+------+--------+



In [76]:
from pyspark.sql.functions import col, isnan
n = df.count()
mask_x = ~isnan(col('clicks')) & col('clicks').isNotNull()
mask_y = ~isnan(col('views')) & col('views').isNotNull()
k = df.filter(mask_x & mask_y & (col('clicks') < col('views'))).count()

In [77]:
cbc_dict = {'total': 10, 'count': 1, 'delta': 1/10}
cbc = CountBelowColumn('views', 'clicks')
assert cbc(df) == cbc_dict

7

In [83]:
df.printSchema()

root
 |-- day: string (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- views: integer (nullable = true)
 |-- clicks: integer (nullable = true)
 |-- payments: integer (nullable = true)

